In [1]:
import json
import os
import random

import decord
import evaluate
from decord import VideoReader, gpu
import numpy as np
import torch
from torchvision import transforms as T
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

decord.bridge.set_bridge('torch')
torch.backends.cuda.matmul.allow_tf32 = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: %s" % device)

Device: cuda


In [2]:
# TimeSformer
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

tokenizer.pad_token = tokenizer.eos_token
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# VideoMAE
# encoder = "MCG-NJU/videomae-base"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained(encoder)
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k600 were not used when initializing TimesformerModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.4.crossattention.masked_bias', 'h.9.crossattention.bias', 'h.8.crossatt

In [3]:
class VatexDataset(torch.utils.data.Dataset):
    def __init__(self, videos_path, json_path, num_frames, transforms=None):
        self.video_names = []
        for _, _, files in os.walk(videos_path):
            for file in files:
                self.video_names.append(file)
        
        self.video_captions = {}
        with open(json_path) as json_file:
            json_data = json.load(json_file)            
            for data in json_data:
                video_id = data["videoID"]
                captions = data["enCap"]
                self.video_captions[video_id] = captions
                
        self.videos_path = videos_path
        self.num_frames = num_frames
        self.transforms = transforms
        

    def __len__(self):
        return len(self.video_names) * 10
    
    
    def __getitem__(self, idx):        
        video_idx = idx % len(self.video_names)
        caption_idx = idx // len(self.video_names)
        
        video_name = self.video_names[video_idx]
        video_id = video_name.split(".")[0]
        captions = self.video_captions[video_id]
        caption = captions[caption_idx]
        
        video_path = os.path.join(self.videos_path, video_name)
        videoreader = VideoReader(video_path, ctx=gpu(0))
        videoreader.seek(0)
        
        indices = np.sort(np.random.uniform(low=0, high=len(videoreader), size=self.num_frames).astype(np.int64))
        #indices = np.linspace(0, len(videoreader) - 1, num=self.num_frames).astype(np.int64)
        frames = videoreader.get_batch(indices)

        return (frames, caption)

In [4]:
num_frames = 8

transforms_train = T.Compose([
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.RandomPerspective(),
    T.RandomRotation(15),
    T.RandomAdjustSharpness(2),
    T.RandomAutocontrast(),
    T.ToTensor(),
    T.Normalize(image_processor.image_mean, image_processor.image_std)
])

transforms_val = T.Compose([
    T.ToTensor(),
    T.Normalize(image_processor.image_mean, image_processor.image_std),
])

data_train = VatexDataset("dataset/vatex_train_videos", "dataset/vatex_train_captions.json", num_frames, transforms_train)
data_val = VatexDataset("dataset/vatex_val_videos", "dataset/vatex_val_captions.json", num_frames, transforms_val)

data_train = torch.utils.data.Subset(data_train, list(range(100)))
data_val = torch.utils.data.Subset(data_val, list(range(10)))

print(len(data_train))
print(len(data_val))

100
10


In [5]:
def collate_fn(examples):
    pixel_values, labels = [], []
    for frames, caption in examples:
        pixel_values.append(image_processor(list(frames), return_tensors="pt").pixel_values)
        labels.append(torch.LongTensor(tokenizer(caption, padding="max_length").input_ids))
    
    pixel_values = torch.cat(pixel_values)
    labels = torch.stack(labels)
    
    return {"pixel_values": pixel_values, "labels": labels}

metric = evaluate.load("rouge")
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=preds, references=labels)

training_args = Seq2SeqTrainingArguments(
    output_dir="trainer",
    tf32=True,
    predict_with_generate=True,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=image_processor,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_val,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/neleac/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 300
  Number of trainable parameters = 274065408
/home/neleac/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:147: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541990/work/torch/

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 11.76 GiB total capacity; 7.97 GiB already allocated; 62.44 MiB free; 8.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
frames, caption = data_train[2]
print(caption)

pixel_values = image_processor(list(frames), return_tensors="pt").pixel_values.to(device)
generated_ids = model.generate(pixel_values)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)